# CRQ2
###### Visualize Taxis movements! NYC is divided in many Taxis zones. For each yellow cab trip we know the zone the Taxi pick up and drop off the users. Let's visualize, on a chropleth map, the number of trips that starts in each zone. Than, do another map to count the races that end up in the single zone. Comment your discoveries. To perform this task we use the library folium. 

In [161]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sys import getsizeof as get

import folium
import json
import branca.colormap as cm
import os
from branca.colormap import linear

In [102]:
M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-01.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-02.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-03.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-04.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-05.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-06.csv", encoding = 'latin-1')

In [103]:
M1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.00,0.0,0.3,8.30
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.00,0.0,0.3,34.80
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55


We need twocolumns. So, we drop the others. keep PULocationID, DOLocationID

In [104]:
M2 = M1.filter(['DOLocationID', 'PULocationID'])
#free the memory from this big object M1 is! 
del M1

At first we want a map with Picking locations. 

In [105]:
pickup=M2.filter(['PULocationID'])
pickup.head()

,PULocationID
0,41
1,239
2,262
3,140
4,246


Let's create a map of NewYork!

In [106]:
NYmap=folium.Map(
    location=[40.7142700, -74.0059700],   #coordinates of new York
    zoom_start=11,                        
    tiles='CartoDB positron'              #style of our map
)
NYmap

Let's add zones to our map. 

In [107]:
zone_geo = os.path.join('data', r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\taxi_zones.json")

geo_json_data = json.load(open(zone_geo))


In [108]:
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': '#00ff00',
        'color': 'black',
        'weight': 0.2,
        'dashArray': '1, 1'
    }
).add_to(NYmap)
#unfortunatly jupyter can't load our map so we store it in a file that we will give separatly
NYmap.save(os.path.join('results', r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\NYCzones.html"))


Now that we know ho to create maps, we want to create a clorphlet map that represents the number of trips that starts in each zone. First thing we need a data frame with counts of taxi pick ups in the zone.

In [124]:
pickup.groupby(['PULocationID']).groups.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 

In [153]:
len(pickup.groupby(['PULocationID']).groups.keys())

259

In [130]:
pickupser=pickup.groupby('PULocationID')['PULocationID'].count()

In [155]:
pickupser.head()

PULocationID
1      571
2        4
3       37
4    19656
5        2
Name: PULocationID, dtype: int64

In [140]:
pickupdf=pd.DataFrame(index=list(range(1,266)),columns=[])

In [142]:
pickupdf['zone']=list(range(1,266))

In [154]:
counts=[]
for i in range(1,266):
    if i in pickupser:
        counts.append(pickupser[i])
    else:
        counts.append(0)     #we need to do this check becouse some zones are missing (maybe 0 taxi taken in that zone)
len(counts)

265

In [158]:
pickupdf['Taxitaken']=counts
pickupdf.head()

,zone,Taxitaken
1,1,571
2,2,4
3,3,37
4,4,19656
5,5,2


Now he have our fixed data frame that we can use to create our fantastic Choropleth maps.


In [209]:
NYmap2 = folium.Map(
    location=[40.7142700, -74.0059700],   #coordinates of new York
    zoom_start=11,                        
    tiles='CartoDB positron'              #style of our map
)

NYmap2.choropleth(
    geo_data=geo_json_data,
    data=pickupdf,
    columns=['zone', 'Taxitaken'],
    key_on='feature.properties.LocationID',
    fill_color='Oranges',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Number of taxi taken in january',
    highlight=True
)
#unfortunatly jupyter can't load our map so we store it in a file that we will give separatly
NYmap2.save(os.path.join('results', r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\NYCChoro.html"))

Now we want to do same job but for drop location.

In [215]:
drops=M2.filter(['DOLocationID'])
dropser=drops.groupby('DOLocationID')['DOLocationID'].count()
dropsdf=pd.DataFrame(index=list(range(1,266)),columns=[])
dropsdf['zone']=list(range(1,266))
counts=[]
for i in range(1,266):
    if i in dropser:
        counts.append(dropser[i])
    else:
        counts.append(0)     #we need to do this check becouse some zones are missing (maybe 0 taxi taken in that zone)
dropsdf['Taxileaved']=counts
pickupdf.head()

,zone,Taxitaken
1,1,571
2,2,4
3,3,37
4,4,19656
5,5,2


In [217]:
NYmap3 = folium.Map(
    location=[40.7142700, -74.0059700],   #coordinates of new York
    zoom_start=11,                        
    tiles='CartoDB positron'              #style of our map
)

NYmap3.choropleth(
    geo_data=geo_json_data,
    data=dropsdf,
    columns=['zone', 'Taxileaved'],
    key_on='feature.properties.LocationID',
    fill_color='Oranges',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Number of taxi drops in january',
    highlight=True
)
#unfortunatly jupyter can't load our map so we store it in a file that we will give separatly
NYmap3.save(os.path.join('results', r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\NYCChoro2.html"))

## RESULTS

As we can appreciate from maps, use of Yellow taxis is highly concentrated in Manhattan. This is true for taxi drops and even more for taxi pickups.
We see that, outside Manhattan, we have two other zones where taxi are used a lot.Predictably, One is "John Fitzgerald Kennedy International", the other one is "La Guardia" airport.